<a href="https://colab.research.google.com/github/molit123/Shakespeare-LSTM/blob/main/shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import matplotlib.pyplot as plt

In [ ]:
from nltk.corpus.reader.cmudict import CMUDictCorpusReader

# Part A - Pre Processing
Here, we will start with word-level tokenization using the CMUDictCorpusReader API from the NLTK Syllable Package.

In [ ]:
#--------------------WORD-LEVEL TOKENIZATION--------------------
with open("drive/MyDrive/CSE 151B/Homework 3/shakespeare.txt", "r") as infile,\
      open("drive/MyDrive/CSE 151B/Homework 3/subset.txt", "w") as outfile:
      text = infile.read()
      text = text.replace("\n\n\n", "")
      outfile.write(text)
corpus = CMUDictCorpusReader(root="drive/MyDrive/CSE 151B/Homework 3", fileids=["subset.txt"])

In [ ]:
words = corpus.words() # Tokenized list of unique words
words

In [ ]:
d = corpus.dict() # Word mapping
d

Since we are going to be training a character-based model, we will use character-level tokenization instead for now.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = open("drive/MyDrive/CSE 151B/Homework 3/shakespeare.txt", 'r').read()
chars = sorted(list(set(data)))
data_size, vocab_size = len(data), len(chars)
print("----------------------------------------")
print("Data has {} characters, {} unique".format(data_size, vocab_size))
print("----------------------------------------")

# char to index and index to char maps
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

----------------------------------------
Data has 98029 characters, 71 unique
----------------------------------------


In [ ]:
# convert data from chars to indices
data = list(data)
for i, ch in enumerate(data):
    data[i] = char_to_ix[ch]

print(data)
data = torch.tensor(data).to(device)
# data = torch.unsqueeze(data, dim=1)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 10, 0, 27, 62, 59, 57, 1, 50, 45, 53, 62, 49, 63, 64, 1, 47, 62, 49, 45, 64, 65, 62, 49, 63, 1, 67, 49, 1, 48, 49, 63, 53, 62, 49, 1, 53, 58, 47, 62, 49, 45, 63, 49, 6, 0, 40, 52, 45, 64, 1, 64, 52, 49, 62, 49, 46, 69, 1, 46, 49, 45, 65, 64, 69, 3, 63, 1, 62, 59, 63, 49, 1, 57, 53, 51, 52, 64, 1, 58, 49, 66, 49, 62, 1, 48, 53, 49, 6, 0, 23, 65, 64, 1, 45, 63, 1, 64, 52, 49, 1, 62, 53, 60, 49, 62, 1, 63, 52, 59, 65, 56, 48, 1, 46, 69, 1, 64, 53, 57, 49, 1, 48, 49, 47, 49, 45, 63, 49, 6, 0, 29, 53, 63, 1, 64, 49, 58, 48, 49, 62, 1, 52, 49, 53, 62, 1, 57, 53, 51, 52, 64, 1, 46, 49, 45, 62, 1, 52, 53, 63, 1, 57, 49, 57, 59, 62, 69, 19, 0, 23, 65, 64, 1, 64, 52, 59, 65, 1, 47, 59, 58, 64, 62, 45, 47, 64, 49, 48, 1, 64, 59, 1, 64, 52, 53, 58, 49, 1, 59, 67, 58, 1, 46, 62, 53, 51, 52, 64, 1, 49, 69, 49, 63, 6, 0, 27, 49, 49, 48, 3, 63, 64, 1, 64, 52, 69, 1, 56, 53, 51, 52, 64, 3, 63, 1, 50, 56, 45, 57, 49, 1, 67, 53, 64, 52, 1, 63, 49,

# Part B - Model Training
Below is the architecture of my model.

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, output_size, hidden_size):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
        self.decoder = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax()

    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

In [ ]:
model = RNN(input_size=vocab_size, embedding_size=vocab_size, output_size=vocab_size, hidden_size=100).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Now, we can go ahead and train the model.

In [ ]:
epochs = 10
perplexities = []

for i_epoch in range(1, epochs+1):

    n = 0
    running_loss = 0

    for i in range(40,len(data)-1):
        hidden_state = None
        input_seq = data[i-40 : i]
        target_seq = data[i-40+1 : i+1]

        # forward pass
        output, _ = model(input_seq, hidden_state)

        # compute loss
        loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
        running_loss += loss.item()
        n += 1

        # compute gradients and take optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # compute perplexity
        #perplexity = torch.exp(loss)
        #perplexities.append(perplexities)

    # print loss after every epoch
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))

# plot the perplexities
#plt.plot(perplexities)

Finally, we can generate the output from our model based on a given prompt.

In [ ]:
prompt = "shall i compare thee to a summer's day?\n"

prompt = list(prompt)
for i, ch in enumerate(prompt):
    prompt[i] = char_to_ix[ch]

with torch.no_grad():
    prompt = torch.tensor(prompt).to(device).long()
    hidden_init = None
    output, hidden = model(prompt, hidden_init)

    for _ in range(160):
        output = output[-1]
        temperature = 0.75
        prediction = torch.multinomial(torch.softmax(output / temperature, dim=0), 1)
        print(ix_to_char[int(prediction.detach().numpy())],end="")
        output, hidden = model(torch.tensor([prediction]), hidden)

# Part C - Improving the model
To improve the model, I decided to experiment with the architecture of the network by increasing the number of LSTM units to 256.

In [ ]:
model = RNN(input_size=vocab_size, embedding_size=vocab_size, output_size=vocab_size, hidden_size=256).to(device)